In [1]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor, nn 

In [39]:
#2 layer classes, forward -> call class, backward -> call .backward() 
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)
#class for MSE Error
class Mse():
    
    def __call__(self, inp, target):
        self.inp = inp
        self.targ = target
        self.out = (inp.squeeze() - target).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2.*(self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [40]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [41]:
MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"

In [42]:
#downloading the mnist dataset from deeplearning.net
path = datasets.download_data(MNIST_URL, ext='.gz')
with gzip.open(path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f,encoding='latin-1')
    
x_train, y_train, x_valid, y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))
n,c = x_train.shape

In [43]:
def normalize(x, m, s): return (x-m)/s

#normalize our data
training_mean, training_std = x_train.mean(), x_train.std()
validation_mean, validation_std = x_valid.mean(),x_valid.std()

x_train = normalize(x_train, training_mean, training_std)
x_valid = normalize(x_train, validation_mean, validation_std)

In [44]:
n, m = x_train.shape
c = y_train.max()+1
n,m,c
# kaiming init / he init for relu
nh = 50
w1 = torch.randn(m,nh)*math.sqrt(2./m )
b1 = torch.zeros(nh)
t1 = relu_subhalf(slope_intercept(x_valid, w1, b1))
print(t1.mean(),t1.std())
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)
w1.g, b1.g, w2.g, b2.g = [None]*4

tensor(1.8508) tensor(2.9602)


In [45]:
model = Model(w1,b1, w2,b2)
%time loss = model(x_train, y_train)

Wall time: 52 ms


In [46]:
%time model.backward()

Wall time: 2.38 s


In [70]:
#improved class with base Class
class Module():
        def __call__(self, *args):
            self.args = args
            self.out = self.forward(*args)
            return self.out
        def forward(self):
            raise Exception('not implemented')
            
        def backward(self):
            self.bwd(self.out, *self.args)
            
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g
        
class Lin(Module):
    def __init__(self, w, b): 
        self.w,self.b = w,b
        
    def forward(self, inp): 
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)
        
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]
        
        
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [71]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [72]:
%time loss = model(x_train, y_train)

Wall time: 46 ms


In [73]:
model.backward()

In [90]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()
#Now Using All Pytorch
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x.squeeze(), targ)

In [91]:
model = Model(m, nh, 1)

In [92]:
%time loss = model(x_train, y_train)

Wall time: 46 ms
